In [1]:
import os
import xml.etree.ElementTree as ET
import random
import shutil
import yaml

def copy_files(file_list, src_dir, dest_dir):
    """
    Copy files from source directory to destination directory based on file list.

    Args:
        file_list (list): List of filenames to copy.
        src_dir (str): Source directory where files are located.
        dest_dir (str): Destination directory where files will be copied.
    """
    os.makedirs(dest_dir, exist_ok=True)  # Ensure destination directory exists
    for file_name in file_list:
        src_path = os.path.join(src_dir, file_name)
        dest_path = os.path.join(dest_dir, file_name)
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
            # print(f"Copied {src_path} to {dest_path}")
        else:
            print(f"File not found: {src_path}")

            
def convert_to_yolo(xml_file, output_dir):
    """
    Convert XML annotations to YOLO format and save to text files.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    cnt = 0
    for image in root.findall("image"):
        image_name = image.attrib["name"]
        image_width = float(image.attrib["width"])
        image_height = float(image.attrib["height"])

        yolo_annotations = []
        for box in image.findall("box"):
            label = box.attrib["label"]
            label = label.upper().replace(" ","").replace("입니다","").replace(".","")
            if (label =="X") or (label =="TRUCK") or (label =="CAR"):
                continue
            xtl = float(box.attrib["xtl"])
            ytl = float(box.attrib["ytl"])
            xbr = float(box.attrib["xbr"])
            ybr = float(box.attrib["ybr"])

            # Convert to YOLO format
            x_center = (xtl + xbr) / 2 / image_width
            y_center = (ytl + ybr) / 2 / image_height
            width = (xbr - xtl) / image_width
            height = (ybr - ytl) / image_height

            # Assuming label IDs:   SUV/sedan/VAN/compact_car                         "TRUCK": 94,
            # class_id = {"SEDAN": 80, "SUV": 81, "VAN" : 82, "COMPACT_CAR" : 83, "BUS": 84}[label]
            class_id = {"SEDAN": 0, "SUV":1, "VAN" : 2, "COMPACT_CAR" : 3, "BUS":4}[label]         
            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # Save YOLO annotations to a text file
        label_file = os.path.join(output_dir, os.path.splitext(image_name)[0] + ".txt")
        with open(label_file, "w") as f:
            f.write("\n".join(yolo_annotations))
            cnt +=1
    print(f"Saved YOLO annotations for {cnt} files")

def check_dataset_integrity(image_dir, label_dir):
    """
    Check if every image in image_dir has a corresponding label in label_dir.
    """
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
    label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    image_names = set(os.path.splitext(f)[0] for f in image_files)
    label_names = set(os.path.splitext(f)[0] for f in label_files)

    missing_labels = image_names - label_names
    if missing_labels:
        # 1==1
        print("Missing labels for images:", len(missing_labels))
    else:
        print("All images have corresponding labels!")


In [3]:
curr_dir

'/app/yolo_structure/notebooks'

In [4]:
curr_dir = "/app/yolo_structure" #os.getcwd()
adjust_xml_dir =  os.path.join(curr_dir, "adjust_label_xml")
origin_img_dir =  os.path.join(curr_dir, "origin_img")
label_txt_dir   = os.path.join(curr_dir, "adjust_label_txt")
if not os.path.exists(label_txt_dir):
    os.makedirs(label_txt_dir)

In [5]:
xml_path = os.path.join(adjust_xml_dir)
xml_list = os.listdir(xml_path)
xml_name_list = []
# 이미지 파일 이름 끝이 .png로 끝나므로 .png로 split 후 이름만 가져온다.
for file_name in xml_list:
    if ".xml" in file_name:
        xml_name_list.append(file_name)
len(xml_name_list)

157

In [6]:
for xml in xml_name_list:
    xml_path =  os.path.join(adjust_xml_dir, xml)
    convert_to_yolo(xml_path, label_txt_dir)

Saved YOLO annotations for 120 files
Saved YOLO annotations for 120 files
Saved YOLO annotations for 112 files
Saved YOLO annotations for 360 files
Saved YOLO annotations for 90 files
Saved YOLO annotations for 123 files
Saved YOLO annotations for 87 files
Saved YOLO annotations for 88 files
Saved YOLO annotations for 120 files
Saved YOLO annotations for 120 files
Saved YOLO annotations for 89 files
Saved YOLO annotations for 120 files
Saved YOLO annotations for 118 files
Saved YOLO annotations for 120 files
Saved YOLO annotations for 116 files
Saved YOLO annotations for 117 files
Saved YOLO annotations for 75 files
Saved YOLO annotations for 113 files
Saved YOLO annotations for 93 files
Saved YOLO annotations for 127 files
Saved YOLO annotations for 68 files
Saved YOLO annotations for 1000 files
Saved YOLO annotations for 109 files
Saved YOLO annotations for 111 files
Saved YOLO annotations for 82 files
Saved YOLO annotations for 111 files
Saved YOLO annotations for 39 files
Saved YOL

KeyError: 'MINIVAN'

In [7]:
# 이미지 저장 폴더(All_Data)에서 이미지 리스트 불러오기
# 이 코드도 1번 코드와 동일한 위치(편의 상 A폴더라고 지칭함)에 있음
Img_path = os.path.join(origin_img_dir)
file_list = os.listdir(Img_path)
file_name_list = []
# 이미지 파일 이름 끝이 .png로 끝나므로 .png로 split 후 이름만 가져온다.
for file_name in file_list:
    if ".png" in file_name:
        file_name_list.append(file_name)
len(file_name_list)

18931

In [8]:
# 랜덤으로 데이터 섞은 후 train, validation 나누기(8:2 비율)
random.shuffle(file_name_list)
Split_Num = int(len(file_name_list) * 0.8)
Train_file_name = file_name_list[:Split_Num]
Valid_file_name = file_name_list[Split_Num:]
print(f"Train : {len(Train_file_name)} / Vaild : {len(Valid_file_name)} ")

Train : 15144 / Vaild : 3787 


In [9]:
yolo_dataset_dir =  os.path.join(curr_dir, "dataset")

In [10]:
# 디렉터리가 존재하면 삭제
if os.path.exists(yolo_dataset_dir):
    shutil.rmtree(yolo_dataset_dir)
    print(f"디렉터리 {yolo_dataset_dir} 삭제 완료.")
else:
    print(f"디렉터리 {yolo_dataset_dir}이(가) 존재하지 않습니다.")

디렉터리 /app/yolo_structure/dataset 삭제 완료.


In [11]:
# Subdirectories
Train_img_path   = os.path.join(yolo_dataset_dir, "images", "train")
Train_label_path = os.path.join(yolo_dataset_dir, "labels", "train")
Valid_img_path   = os.path.join(yolo_dataset_dir, "images", "valid")
Valid_label_path = os.path.join(yolo_dataset_dir, "labels", "valid")


# List of directories to check and create
directories = [Train_img_path, Train_label_path, Valid_img_path, Valid_label_path]

# Create directories if they don't exist
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")
    else:
        print(f"Directory already exists: {directory}")

Created directory: /app/yolo_structure/dataset/images/train
Created directory: /app/yolo_structure/dataset/labels/train
Created directory: /app/yolo_structure/dataset/images/valid
Created directory: /app/yolo_structure/dataset/labels/valid


In [12]:
# Copy valid and train files
copy_files(Valid_file_name, origin_img_dir, Valid_img_path)
copy_files(Train_file_name, origin_img_dir, Train_img_path)

In [13]:
valid_label_txt_l = []
for txt_label_file in os.listdir(label_txt_dir):
    for target_name in Valid_file_name:
        if target_name.split('.')[0] in txt_label_file:
            valid_label_txt_l.append(txt_label_file)
            
train_label_txt_l = []
for txt_label_file in os.listdir(label_txt_dir):
    for target_name in Train_file_name:
        if target_name.split('.')[0] in txt_label_file:
            train_label_txt_l.append(txt_label_file)
print(f"Train text length : {len(train_label_txt_l)} , Validation text length : {len(valid_label_txt_l)}")

Train text length : 13218 , Validation text length : 3308


In [14]:
# Copy valid and train files
copy_files(valid_label_txt_l, label_txt_dir, Valid_label_path)
copy_files(train_label_txt_l, label_txt_dir, Train_label_path)

In [15]:
check_dataset_integrity(Train_img_path, Train_label_path)

Missing labels for images: 1926


In [16]:
print(f"Train 이미지 개수: {len([f for f in os.listdir(Train_img_path) if f.endswith(('.jpg', '.png'))])}")
print(f"Train 라벨 개수: {len([f for f in os.listdir(Train_label_path) if f.endswith('.txt')])}")
print(f"Val 이미지 개수: {len([f for f in os.listdir(Valid_img_path) if f.endswith(('.jpg', '.png'))])}")
print(f"Val 라벨 개수: {len([f for f in os.listdir(Valid_label_path) if f.endswith('.txt')])}")

Train 이미지 개수: 15144
Train 라벨 개수: 13218
Val 이미지 개수: 3787
Val 라벨 개수: 3308


## Making yaml

In [17]:
import xml.etree.ElementTree as ET
def extract_unique_labels(xml_file):
    """
    Extract unique 'label' values from <box> elements in the given XML file.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Set to store unique labels
    unique_labels = set()
    
    # Find all <box> elements
    for box in root.findall(".//box"):
        label = box.attrib.get("label")
        if label:
            if (label =="X") or (label =="TRUCK") or (label =="CAR") :
                continue
            label = label.replace("'",'').replace("입니다","").replace(".","").upper()
            unique_labels.add(label)
    return unique_labels

def get_all_unique_labels(meta_list):
    """
    Extract all unique labels from a list of XML files.
    """
    unique_label_set = set()  # Use a set for efficient deduplication
    for xml_file in meta_list:
        unique_label_set.update(extract_unique_labels(xml_file))
    return list(unique_label_set)


In [18]:
meta_list = []
for file_name in  os.listdir(adjust_xml_dir):
    if (".xml" in file_name):
        meta_list.append(os.path.join(adjust_xml_dir,file_name) )
    break
meta_list

new_labels = get_all_unique_labels(meta_list)
new_labels

['SUV', 'TRUCK', 'BUS', 'VAN', 'COMPACT_CAR', 'SEDAN']

In [19]:
from ultralytics import YOLO

In [21]:
curr_dir

'/app/yolo_structure'

In [22]:
model = YOLO(curr_dir + "/yolo11x.pt") # 원하는 크기 모델 입력(n, s, m, l, x 등 원하는 모델을 로드함)
print(type(model.names),len(model.names))
print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [23]:
# 기존 라벨 값들을 리스트로 변환
existing_labels_list = list(model.names.values())
# 기존 라벨과 새로운 라벨을 결합하여 새로운 리스트 생성
all_labels = existing_labels_list + new_labels
all_labels

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush',
 'SUV',
 'TRUCK',
 'BUS',
 'VAN',
 'COMPACT_CAR',
 'SEDAN']

In [24]:
yaml_data = {'train': Train_img_path, 'val': Valid_img_path, 
          'nc': len(new_labels), 'names': new_labels}

yaml_path = os.path.join(curr_dir,'data.yaml')

with open(yaml_path,'w') as f:
    yaml.dump(yaml_data,f)

with open(yaml_path,'r') as f:
    yaml_file = yaml.load(f, Loader=yaml.FullLoader)
    display(yaml_file)

{'names': ['SUV', 'TRUCK', 'BUS', 'VAN', 'COMPACT_CAR', 'SEDAN'],
 'nc': 6,
 'train': '/app/yolo_structure/dataset/images/train',
 'val': '/app/yolo_structure/dataset/images/valid'}

In [25]:
yaml_path = os.path.join(curr_dir,'data.yaml')
yaml_path

'/app/yolo_structure/data.yaml'

In [ ]:
test_loc = os.path.join(curr_dir, "test")  # 테스트 데이터셋 경로
output_dir = os.path.join(curr_dir, "case_results")  # 결과 저장 경로
os.makedirs(output_dir, exist_ok=True)

# 테스트 설정
freeze_values = [0, 10, 25, 50]
epoch_values = [50, 100, 200]
model_path = curr_dir + "/" + "yolo11x.pt"
# 결과 저장용 리스트
results_summary = []

# 테스트 루프
for freeze in freeze_values:
    for epochs in epoch_values:
        # 모델 훈련
        print(f"Training: freeze={freeze}, epochs={epochs}")
        model = YOLO(model_path)
        model.train(
            data=yaml_path,
            epochs=epochs,
            freeze=freeze,
            batch=16,
            workers=4,
            imgsz=640
        )

        # 예측 및 결과 저장
        save_path = os.path.join(output_dir, f"freeze_{freeze}_epochs_{epochs}")
        os.makedirs(save_path, exist_ok=True)
        print(f"Predicting for freeze={freeze}, epochs={epochs}")
        results = model.predict(source=test_loc, save=True, save_dir=save_path)

        # 결과 요약
        results_summary.append({
            "freeze": freeze,
            "epochs": epochs,
            # "mAP50": results.metrics.mAP50,
            # "mAP50-95": results.metrics.mAP50_95,
            "images_saved": len(results.files)
        })

# 결과 요약 출력
import pandas as pd
df = pd.DataFrame(results_summary)
df.sort_values(by=["mAP50", "mAP50-95"], ascending=False, inplace=True)
print(df)

# 결과 저장
df.to_csv(os.path.join(output_dir, "summary_results.csv"), index=False)


Training: freeze=0, epochs=50
Ultralytics 8.3.70 🚀 Python-3.12.2 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15966MiB)
engine/trainer: task=detect, mode=train, model=/app/yolo_structure/yolo11x.pt, data=/app/yolo_structure/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=0, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_con

100%|██████████████████████████████████████████████████████████████| 755k/755k [00:01<00:00, 407kB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

 20                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
 21            [-1, 10]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 22                  -1  2   5612544  ultralytics.nn.modules.block.C3k2            [1536, 768, 2, True]          
 23        [16, 19, 22]  1   3152482  ultralytics.nn.modules.head.Detect           [6, [384, 768, 768]]          
YOLO11x summary: 631 layers, 56,880,706 parameters, 56,880,690 gradients, 195.5 GFLOPs

Transferred 1009/1015 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|████████████████████████████████████████████████████████████| 5.35M/5.35M [00:08<00:00, 630kB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/50      15.8G     0.8358      1.089     0.8575         40        640: 100%|██████████| 947/9
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████

                   all       3787       9981      0.462      0.519      0.478      0.388



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/50      15.8G      0.783      1.032     0.8493        108        640:  84%|████████▎ | 793/9

In [42]:
results = model.predict(source=test_loc, save=True, save_dir=save_path)


image 1/3 /app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png: 640x384 (no detections), 7.6ms
image 2/3 /app/yolo_structure/test/hh_dash.jpg: 384x640 (no detections), 7.5ms
image 3/3 /app/yolo_structure/test/people_street.jpeg: 384x640 (no detections), 7.3ms
Speed: 0.7ms preprocess, 7.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict2


In [38]:
save_path

'/app/yolo_structure/case_results/freeze_0_epochs_50'

In [40]:
# 결과 요약
results_summary.append({
    "freeze": freeze,
    "epochs": epochs,
    # "mAP50": results.metrics.mAP50,
    # "mAP50-95": results.metrics.mAP50_95,
    "images_saved": save_path
})

# 결과 요약 출력
import pandas as pd
df = pd.DataFrame(results_summary)
# df.sort_values(by=["mAP50", "mAP50-95"], ascending=False, inplace=True)
print(df)

# 결과 저장
df.to_csv(os.path.join(output_dir, "summary_results.csv"), index=False)


   freeze  epochs                                       images_saved
0       0      50  /app/yolo_structure/case_results/freeze_0_epoc...
1       0      50  /app/yolo_structure/case_results/freeze_0_epoc...


In [31]:
# 학습 데이터에 기존 및 새 데이터 포함
model.train(data=yaml_path,  # 기존 + 새 데이터셋 경로
            epochs=500, 
            patience=10 , augment=True, 
            imgsz=640, 
            batch=16, 
            workers=4,
           freeze=10)  


Ultralytics 8.3.67 🚀 Python-3.12.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15966MiB)
engine/trainer: task=detect, mode=train, model=yolo11x.pt, data=/app/yolo_structure/data.yaml, epochs=500, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

train: Scanning /app/yolo_structure/dataset/labels/train... 8394 images, 3150 backgrounds, 0 corrupt: 100%|██████████| 1020


train: New cache created: /app/yolo_structure/dataset/labels/train.cache


val: Scanning /app/yolo_structure/dataset/labels/valid... 2069 images, 822 backgrounds, 0 corrupt: 100%|██████████| 2552/25

val: New cache created: /app/yolo_structure/dataset/labels/valid.cache


...

                   all       2552       5864      0.634      0.612      0.635      0.553
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 48, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



58 epochs completed in 2.665 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 114.4MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics 8.3.67 🚀 Python-3.12.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15966MiB)
YOLO11x summary (fused): 464 layers, 56,833,954 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 80/80 [00:28<00:


                   all       2552       5864      0.608      0.625      0.638      0.548
                   BUS       1441       4007       0.66      0.823      0.777      0.647
                   VAN        855       1249      0.581      0.637      0.652      0.573
                   SUV        179        185      0.543      0.524      0.532      0.477
           COMPACT_CAR         83         83      0.457      0.294      0.354      0.316
                 TRUCK        289        340      0.801      0.844      0.877      0.728
Speed: 0.0ms preprocess, 7.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x764e08634ce0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [32]:
test_loc = os.path.join(curr_dir,"test")
results = model.predict(source = test_loc,save=True)


image 1/3 /app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png: 640x384 17 BUSs, 9 VANs, 67.1ms
image 2/3 /app/yolo_structure/test/hh_dash.jpg: 384x640 (no detections), 64.5ms
image 3/3 /app/yolo_structure/test/people_street.jpeg: 384x640 1 BUS, 19.7ms
Speed: 0.8ms preprocess, 50.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/train52


In [29]:
test_loc = os.path.join(curr_dir,"test")
results = model.predict(source = test_loc,save=True)


image 1/3 /app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png: 640x384 6 BUSs, 7 COMPACT_CARs, 1 SEDAN, 7.8ms
image 2/3 /app/yolo_structure/test/hh_dash.jpg: 384x640 (no detections), 7.5ms
image 3/3 /app/yolo_structure/test/people_street.jpeg: 384x640 (no detections), 7.2ms
Speed: 0.8ms preprocess, 7.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/train432


In [26]:
# 학습 데이터에 기존 및 새 데이터 포함
model.train(data=yaml_path,  # 기존 + 새 데이터셋 경로
            epochs=100, 
            patience=30, 
            imgsz=640, 
            batch=16, 
            workers=4)  


Ultralytics 8.3.67 🚀 Python-3.12.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15966MiB)
engine/trainer: task=detect, mode=train, model=yolo11x.pt, data=/app/yolo_structure/data.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

100%|███████████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 1.43MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

  8                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  9                  -1  1   1476864  ultralytics.nn.modules.block.SPPF            [768, 768, 5]                 
 10                  -1  2   3264768  ultralytics.nn.modules.block.C2PSA           [768, 768, 2]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  2   5612544  ultralytics.nn.modules.block.C3k2            [1536, 768, 2, True]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  2   1700352  ultralytics.nn.modules.block.C3k2            [1536

train: Scanning /app/yolo_structure/dataset/labels/train... 5379 images, 926 backgrounds, 0 corrupt: 100%|██████████| 5379/

...

                   all       1345       3734      0.643      0.615      0.625      0.552



100 epochs completed in 3.158 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 114.4MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 114.4MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.67 🚀 Python-3.12.2 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15966MiB)
YOLO11x summary (fused): 464 layers, 56,833,954 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:13<00:


                   all       1345       3734       0.62      0.658       0.66      0.579
                   BUS        951       2680      0.706      0.856      0.816      0.712
           COMPACT_CAR        504        726      0.556      0.718      0.698      0.625
                 SEDAN         86         92      0.529      0.538      0.533      0.484
                   SUV         38         39      0.467      0.333      0.363       0.32
                 TRUCK        174        197      0.841      0.843      0.889      0.755
Speed: 0.0ms preprocess, 3.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x775ed4104bc0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        


image 1/3 /app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png: 640x384 6 BUSs, 7 COMPACT_CARs, 1 SEDAN, 21.2ms
image 2/3 /app/yolo_structure/test/hh_dash.jpg: 384x640 (no detections), 21.1ms
image 3/3 /app/yolo_structure/test/people_street.jpeg: 384x640 (no detections), 7.2ms
Speed: 0.6ms preprocess, 16.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/train42



image 1/3 /app/yolo_structure/test/Suwon_CH01_20200721_1500_TUE_9m_NH_highway_TW5_sunny_FHD_013.png: 640x384 11 TRUCKs, 5 SEDANs, 1 VAN, 29.6ms
image 2/3 /app/yolo_structure/test/hh_dash.jpg: 384x640 1 COMPACT_CAR, 26.9ms
image 3/3 /app/yolo_structure/test/people_street.jpeg: 384x640 (no detections), 7.3ms
Speed: 3.3ms preprocess, 21.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/train1132


In [33]:
model_save_path = os.path.join(curr_dir, "my_test_model.pt")  # 모델 저장 경로
model.save(model_save_path)  # 학습된 모델 저장

In [24]:
import torch
print(torch.__version__)  # PyTorch 버전
print(torch.version.cuda)  # CUDA 버전


2.5.1+cu124
12.4


In [33]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
1
NVIDIA GeForce RTX 4080 SUPER
